In [415]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
# import xmltojson
import json
import requests

In [416]:
def range_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    lowend = soup.find_all("span", {"class":"value", "itemprop":"lowprice", "content": True})
    highend = soup.find_all("span", {"class":"value", "itemprop":"highprice", "content": True})
    quantityList = []
    rangePrice = ""

    for i in lowend:
        rangePrice+=(i["content"])
        rangePrice+=" - "
        break

    for i in highend:
        rangePrice+=(i["content"])
        break

    return rangePrice


In [417]:
def scrape_handguns_helper(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    url+= "=undefined&start=0&sz=100"
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    ammo = soup.find_all("div", class_="pdp-link")
    ammoPrice = soup.find_all("div",  class_="tile-body")
    names = []
    prices = []
    actual_prices = []
    for i in ammo:
        names.append(i.find("span").text)

    for i in ammoPrice:
        # if the product has a range of prices go here
        if '<span class="range">' in str(i):
            x = i.find("a", {"class": "link", "href":True})
            prices.append(x["href"])

        # if the product has a slashed price, go here
        else:
            x = (i.find("span", class_="sales"))
            prices.append(x.find("span", {"class":"value", "content": True}))


    # add prices in the prices list
    for i in prices:
        if "http" in i:
            rp = range_price(i)
            actual_prices.append(rp)

        else:
            htmlPriceTag = str(i)
            splitHtmlPriceTag = htmlPriceTag.split(" ")
            p =  splitHtmlPriceTag[2].strip('content="')
            actual_prices.append(str(p))

   
    try:
        productList = pd.DataFrame(
            {"name": names, "price": actual_prices}
        )
        return productList

    except:
        print("something went wrong")

In [418]:
# https://www.cheaperthandirt.com/9mm-luger-parabellum/ammunition/handgun-ammo/?https://www.cheaperthandirt.com/9mm-luger-parabellum/ammunition/handgun-ammo/=undefined&start=0&sz=36
handgunByCaliberLink = "https://www.cheaperthandirt.com/shop-by?cgid=78&searchBy=Caliber"
def scrape_handguns(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    sections = soup.find_all("ul",class_= "col-lg-11 right-list")
    dirtyLinks = []  
    calibers = []  # names
    linkToCaliberList = []  # links
    
    for i in sections:
        dirtyLinks.append(i.find_all("a"))

    for i in dirtyLinks:
        for j in i:
            aLink = re.findall("(?P<url>https?://[^\s]+)", str(j))
            linkToCaliberList.append(str(aLink[0]).replace('">', ""))
            calibers.append(j.find("span").text)


    handgunCaliberLinks = pd.DataFrame(
        {"caliber": calibers, "links": linkToCaliberList}
    )

    productList = []
    for l in handgunCaliberLinks["links"]:
        productList.append(scrape_handguns_helper(l))
        
    return productList

In [419]:
handgunDF = (scrape_handguns(handgunByCaliberLink))

In [443]:
from pathlib import Path
import os, shutil, glob
rel = "jsonFiles/cheaperThanDirt/ammo"
cwd = Path.cwd()
savePath = str(cwd).replace("pyScripts", rel)
currentPath = str(Path.cwd())+""
aCount = 0


for i in handgunDF:
    result = i.to_json(orient="split")
    parsed = json.loads(result)
    data = json.dumps(parsed, indent=4)
    with open(f"{aCount}.json", "w") as fp:
        json.dump(data, fp)

    currentPath = Path.cwd()
    x = (str(currentPath)+f"/{aCount}.json")

    try:
        shutil.move(x, savePath)
    except:
        pass
    aCount += 1

Error: Destination path '/Users/chakaneshegog/Desktop/crispy-computing-machine/jsonFiles/cheaperThanDirt/ammo/0.json' already exists